# Step 5: RLHF - 人类偏好对齐

## 学习目标

1. 理解人类偏好对齐的目标
2. 理解 DPO 原理
3. **动手实现** DPO Loss

## 核心问题

SFT 后模型能对话，但回答质量参差不齐。如何让模型输出更符合人类偏好？

---

## 1. 什么是人类偏好？

给定同一个问题，人类会偏好某些回答：

```
问题: "什么是AI？"

回答 A (chosen): "AI是人工智能的缩写，是计算机科学的一个分支..."
回答 B (rejected): "AI就是机器人。"

人类偏好: A > B
```

---

## 2. DPO vs PPO

### 传统 RLHF (PPO)

```
偏好数据 → 训练 Reward Model → PPO 训练 → 对齐模型
```

问题：需要额外的 Reward Model，PPO 训练不稳定

### DPO (Direct Preference Optimization)

```
偏好数据 → 直接优化 → 对齐模型
```

优点：无需 Reward Model，更简单稳定

---

## 3. DPO 核心公式

$$L_{DPO} = -\log \sigma\left(\beta \cdot \left(\log \frac{\pi(y_w|x)}{\pi_{ref}(y_w|x)} - \log \frac{\pi(y_l|x)}{\pi_{ref}(y_l|x)}\right)\right)$$

其中：
- $y_w$: chosen（好回答）
- $y_l$: rejected（差回答）
- $\pi$: 策略模型（训练中）
- $\pi_{ref}$: 参考模型（冻结）
- $\beta$: 温度系数

### 直觉理解

1. 增加 chosen 的概率
2. 降低 rejected 的概率
3. 不要偏离参考模型太远

In [ ]:
import torch
import torch.nn.functional as F

# DPO Loss 演示
def demo_dpo_loss(policy_chosen, policy_rejected, ref_chosen, ref_rejected, beta=0.1):
    """DPO Loss 的计算过程"""
    
    # Step 1: 策略模型的 log ratio
    pi_logratios = policy_chosen - policy_rejected
    print(f"策略模型 log ratio: {pi_logratios.item():.4f}")
    print(f"  → 正值表示策略模型更偏好 chosen")
    
    # Step 2: 参考模型的 log ratio  
    ref_logratios = ref_chosen - ref_rejected
    print(f"\n参考模型 log ratio: {ref_logratios.item():.4f}")
    
    # Step 3: 相对改进
    logits = pi_logratios - ref_logratios
    print(f"\n相对改进: {logits.item():.4f}")
    print(f"  → 正值表示策略模型比参考模型更偏好 chosen")
    
    # Step 4: Loss
    loss = -F.logsigmoid(beta * logits)
    print(f"\nDPO Loss: {loss.item():.4f}")
    
    return loss

# 示例
print("=" * 50)
print("DPO Loss 计算过程")
print("=" * 50)

demo_dpo_loss(
    policy_chosen=torch.tensor(-1.0),    # 策略模型: chosen 概率较高
    policy_rejected=torch.tensor(-2.0),  # 策略模型: rejected 概率较低
    ref_chosen=torch.tensor(-1.5),       # 参考模型: 两者差异不大
    ref_rejected=torch.tensor(-1.6),
    beta=0.1
)

### 3.1 练习：实现 DPO Loss

去 `train_dpo_exercise.py`，完成 **TODO 1**

In [ ]:
# 测试你的实现
import importlib
import train_dpo_exercise
importlib.reload(train_dpo_exercise)

train_dpo_exercise.test_dpo_loss()

---

## 4. 验证清单

- [ ] 解释 RLHF 的目标（让模型输出符合人类偏好）
- [ ] 解释 DPO 相比 PPO 的优势（无需 Reward Model）
- [ ] 实现 DPO Loss
- [ ] 理解 β 参数的作用（控制偏离程度）

---

## 下一步

进入 [Step 6: VLM](../step6_vlm/)，学习如何扩展到多模态。